In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/자기주도연구

/content/drive/MyDrive/자기주도연구


In [ ]:
!ls

'comments_나의 완벽한 비서_keyword.csv'   comments_중증외상센터_keyword.csv  '댓글 정리.ipynb'
 comments_모텔캘리포니아_keyword.csv	  filtering.ipynb		     '데이터 수집'
 comments_오징어게임2_keyword.csv	 '관련없는 댓글 삭제(전처리)'	     '데이터 통계.ipynb'
 comments_옥씨부인전_keyword.csv	 '긍정부정 이유(keyword).ipynb'      '세부적 이유 추출'
 comments_원경_keyword.csv		 '댓글데이터 수집.ipynb'	     '언급횟수 그래프'


In [ ]:
from openai import OpenAI
import json

In [ ]:
com1 = pd.read_csv('세부적 이유 추출/캐스팅/나의 완벽한 비서/positive/나의 완벽한 비서_casting_positive_reason.csv')
com2 = pd.read_csv('세부적 이유 추출/캐스팅/모텔캘리포니아/positive/모텔캘리포니아_casting_positive_reason.csv')
com3 = pd.read_csv('세부적 이유 추출/캐스팅/오징어게임2/positive/오징어게임2_casting_positive_reason.csv')
com4 = pd.read_csv('세부적 이유 추출/캐스팅/옥씨부인전/positive/옥씨부인전_casting_positive_reason.csv')
com5 = pd.read_csv('세부적 이유 추출/캐스팅/원경/positive/원경_casting_positive_reason.csv')
com6 = pd.read_csv('세부적 이유 추출/캐스팅/중증외상센터/positive/중증외상센터_casting_positive_reason.csv')

In [ ]:
comment_list = pd.concat([com1, com2, com3, com4, com5, com6], ignore_index=True)

In [ ]:
# 유사 or 동일한 카테고리는 하나로 묶기
def clear_actor(col1, col2):
  global comment_list
  plus_comment = comment_list.loc[col1,"reason"]+comment_list.loc[col2,"reason"]
  comment_list.at[col1,"reason"] = plus_comment
  plus_count = comment_list.loc[col1,"count"]+comment_list.loc[col2,"count"]
  comment_list.at[col1,"count"] = plus_count

## 캐스팅 세부 요소별 count

In [ ]:
comment_list

,actor,reason,count
0,메인커플 케미(조합),['both are just perfect with perfect chemistry...,23
1,김도훈,"[nan, nan, nan, '무빙때는 완전 학생같더니 여기서는 또 완전 직장인 같음']",4
2,배우들,"['케미 넘사벽', nan, '얼굴', '연기력', '원래 잘난 배우들인데 더더 돋...",7
3,아역들,['넘귀욥'],1
5,이상희,"['연기 정말 잘하심', nan, '연기를 진짜 잘함']",3
...,...,...,...
60,주지훈,"['피지컬이 미쳤음', nan, '연기력 최고', '너무 멋진 사이다역 찰떡임', ...",143
61,주지훈 추영우,"[nan, nan, '케미 말해 뭐혀요', '캐미 넘 좋아요']",4
62,주지훈 윤경호,['둘의 케미가 빵빵 터지게 함'],1
63,추영우,"[nan, '졸귀', nan, '연기 천재들입니다', '연기 스타일도 먼가 비슷하고...",153


In [ ]:
comment_list.to_csv("all_positive_reason.csv", index=False)

In [ ]:
comment_list = pd.read_csv("all_positive_reason.csv")

In [ ]:
comment_list

,actor,reason,count
0,메인커플 케미(조합),['both are just perfect with perfect chemistry...,23
1,김도훈,"[nan, nan, nan, '무빙때는 완전 학생같더니 여기서는 또 완전 직장인 같음']",4
2,배우들,"['케미 넘사벽', nan, '얼굴', '연기력', '원래 잘난 배우들인데 더더 돋...",7
3,아역들,['넘귀욥'],1
4,이상희,"['연기 정말 잘하심', nan, '연기를 진짜 잘함']",3
...,...,...,...
57,주지훈,"['피지컬이 미쳤음', nan, '연기력 최고', '너무 멋진 사이다역 찰떡임', ...",143
58,주지훈 추영우,"[nan, nan, '케미 말해 뭐혀요', '캐미 넘 좋아요']",4
59,주지훈 윤경호,['둘의 케미가 빵빵 터지게 함'],1
60,추영우,"[nan, '졸귀', nan, '연기 천재들입니다', '연기 스타일도 먼가 비슷하고...",153


In [ ]:
def generate_summary(trend_data):
    client = OpenAI(
      api_key = "my_gpt_api_key"
    )

    """GPT를 사용하여 캐스팅 요소별 트렌드 요약 생성"""
    prompt = f"""
    다음은 시청자들의 반응을 포함하는 데이터를 배우별로 정리해둔 데이터셋이다. 이 데이터를 기반으로 K-드라마에서 사람들이 어떤 부분을 선호하고, 긍정적인 반응을 보이는지 트렌드를 분석하세요.

    {trend_data.to_string(index=False)}

    분석할 내용:
    1. 시청자들이 선호하는 부분
    2. 전반적인 감상 트렌드
    3. 앞으로 드라마 시장에서 계속 반영되었으면 하는 부분

    형식:
    - **주요 트렌드 요약**: (트렌드 내용)
    - **시청자들이 선호하는 요소**: (예시)
    - **기타 주목할 만한 사항**: (예시)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 드라마 캐스팅 분석 전문가야."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content.strip()


In [ ]:
comment_list1 = comment_list[0:25]
comment_list2 = comment_list[25:50]
comment_list3 = comment_list[50:len(comment_list)]

In [ ]:
trend_summary1 = generate_summary(comment_list1)

In [ ]:
trend_summary2 = generate_summary(comment_list2)

In [ ]:
trend_summary3 = generate_summary(comment_list3)

In [ ]:
trend_summary = [trend_summary1, trend_summary2, trend_summary3]

In [ ]:
def generate_final_summary(trend_data):
    client = OpenAI(
      api_key = "my_gpt_api_key"
    )

    """GPT를 사용하여 캐스팅 요소에 대한 트렌드 요약 생성"""

    summaries_text = "\n\n".join(trend_data)

    prompt = f"""
    다음은 시청자들의 반응을 포함하는 데이터셋을 바탕으로 분석한 결과이다. 이 결과들을 바탕으로 K-드라마에서 사람들이 어떤 부분을 선호하고, 긍정적인 반응을 보이는지 트렌드를 분석하세요.

    {summaries_text}

    분석할 내용:
    1. 시청자들이 선호하는 부분
    2. 전반적인 감상 트렌드
    3. 앞으로 드라마 시장에서 계속 반영되었으면 하는 부분

    형식:
    - **주요 트렌드 요약**: (트렌드 내용)
    - **시청자들이 선호하는 요소**: (예시)
    - **기타 주목할 만한 사항**: (예시)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 드라마 캐스팅 분석 전문가야."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content.strip()


In [ ]:
final_summary_trend = generate_final_summary(trend_summary)

In [ ]:
with open("casting_positive_trends_summary.md", "w", encoding="utf-8") as f: #appned : w->a
    f.write("드라마 최신 트랜드 분석(캐스팅 - positive)")  # 기존 내용과 구분
    f.write(final_summary_trend)

print("트렌드 요약이 casting_positive_trends_summary.md 파일에 추가되었습니다.")

트렌드 요약이 casting_positive_trends_summary.md 파일에 추가되었습니다.
